## Modelling notebook

### Group 8 Members
#### Spring Semester 2024-2025
- Alexandre Gonçalves - 20240738
- Bráulio Damba - 20240007
- Hugo Fonseca - 20240520
- Ricardo Pereira - 20240745
- Victoria Goon - 20240550

## 0 - Imports

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import scale
from sklearn.model_selection import GridSearchCV
import re
import string
import nltk
from sklearn.metrics.pairwise import cosine_distances
from collections import Counter
from nltk.corpus import stopwords
from wordcloud import WordCloud
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import re
import string

from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer

# Text extraction 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_distances
from collections import Counter

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from gensim.models import FastText

import scipy.sparse
from scipy import sparse

import contractions
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import gensim.downloader as api

from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.utils import simple_preprocess

import pickle
from sklearn.utils import resample

from itertools import product

from sentence_transformers import SentenceTransformer

# Deep Learning libraries
from keras.models import Sequential,Model
from keras.layers import Dense, Activation, Dropout, Flatten, Input
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, LSTM, Bidirectional, Dropout, Flatten, GRU
from tensorflow.keras.optimizers import Adam
from transformers import AutoTokenizer, AutoModel
import torch

# Set pd options to display all columns and rows
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 30)
pd.set_option('display.max_colwidth', None)  # Show full text without truncation

import glob
from scipy.sparse import load_npz

from tensorflow.keras.layers import Layer
import tensorflow as tf
from tensorflow.keras.layers import Input, Bidirectional, GRU, Flatten, Dense

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
#from scikeras.wrappers import KerasClassifier
from sklearn.base import clone

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import Dataset

#import keras_tuner as kt
from tensorflow.keras import backend as K


# Download required resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="xgboost")

[nltk_data] Downloading package punkt to /Users/ricardo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ricardo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ricardo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/ricardo/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Import the encoded files

In [5]:
# Define the base directory (where the notebook is)
BASE_DIR = os.path.dirname(os.path.abspath("__file__"))

# Construct full paths to the CSV files
train_path = os.path.join(BASE_DIR, "data", "train.csv")
test_path = os.path.join(BASE_DIR, "data", "test.csv")

# Load the datasets
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [6]:
folder = '.'  # Use '.' if running in the current directory, or the path to your folder

# List all .npy and .npz files
npy_files = glob.glob(os.path.join(folder, '*.npy'))
npz_files = glob.glob(os.path.join(folder, '*.npz'))


In [7]:
npy_embeddings = {}

for f in npy_files:
    name = os.path.splitext(os.path.basename(f))[0]
    npy_embeddings[name] = np.load(f, allow_pickle=True)

In [ ]:
npz_embeddings = {}

for f in npz_files:
    name = os.path.splitext(os.path.basename(f))[0]
    try:
        npz_embeddings[name] = load_npz(f)
    except Exception as e:
        print(f"Warning: Failed to load '{f}' as a sparse matrix. Error: {e}")
        try:
            npz_embeddings[name] = np.load(f)
            print(f"Loaded '{f}' as dense array (np.load).")
        except Exception as e2:
            print(f"Failed to load '{f}' as dense array too. Skipping. Error: {e2}")


In [9]:
print(npy_embeddings.keys())  
print(npz_embeddings.keys()) 


dict_keys(['minilm_l12_v2_text_no_lemma_stem_with_stopwords_test', 'w2v_seq_text_no_lemma_stem_with_stopwords_train', 'val_text_no_lemma_stem_with_stopwords', 'distilroberta_v1_text_no_lemma_stem_with_stopwords_train', 'glove_seq_text_no_lemma_stem_with_stopwords_val', 'mpnet_base_v2_text_no_lemma_stem_with_stopwords_val', 'glove_seq_text_no_lemma_stem_with_stopwords_test', 'ft_seq_text_no_lemma_stem_with_stopwords_test', 'mpnet_base_v2_text_no_lemma_stem_with_stopwords_train', 'ft_seq_text_no_lemma_stem_with_stopwords_val', 'all_minilm_l6_v2_text_no_lemma_stem_with_stopwords_test', 'ft_seq_text_no_lemma_stem_with_stopwords_train', 'all_minilm_l6_v2_text_no_lemma_stem_with_stopwords_train', 'glove_seq_text_no_lemma_stem_with_stopwords_train', 'all_minilm_l6_v2_text_no_lemma_stem_with_stopwords_val', 'train_text_no_lemma_stem_with_stopwords', 'minilm_l12_v2_text_no_lemma_stem_with_stopwords_train', 'w2v_seq_text_no_lemma_stem_with_stopwords_val', 'w2v_seq_text_no_lemma_stem_with_stopwor

We perform the split again with the same random state, in order to save space, as we don't need to save our y_train, y_val and y_test, and we can simply get it from the reproducible splits.

In [10]:
# Using stratify to maintain the distribution of classes in the train, validation, and test sets 
# As our dataset is quite small, we use 80% for training, and split the remaining 20% into validation and test sets (10% each).

train_df, val_test_df = train_test_split(df_train, test_size=0.2, stratify=df_train['label'], random_state=42)
val_df, test_df = train_test_split(val_test_df, test_size=0.5, stratify=val_test_df['label'], random_state=42)

In [11]:
y_train = train_df['label']
y_val = val_df['label']
y_test = test_df['label']

## 1 - Model training

### 1.1 - Traditional ML Models with Bag of Words and TF-IDF

In [ ]:
models = {
    "SVC": SVC(class_weight='balanced', random_state=42),  
    "XGB": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=300, class_weight='balanced', random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}


In [ ]:
statistical_methods_results = []

for variant in npz_embeddings:
    if not (variant.startswith("bow") or variant.startswith("tfidf")):
        continue
    
    if not variant.endswith("_train"):
        continue  
    
    base = variant[:-6]  
    train_X = npz_embeddings[variant]
    val_X = npz_embeddings.get(base + "_val")
    if val_X is None:
        print(f"Missing val split for {base}")
        continue
    
    print(f"Training on {base}...")
    for name, model in models.items():
        # Fit model
        model.fit(train_X, y_train)
        y_pred = model.predict(val_X)
        
        # Metrics
        report = classification_report(y_val, y_pred, output_dict=True)
        statistical_methods_results.append({
            "variant": base,
            "model": name,
            "accuracy": report["accuracy"],
            "macro_f1": report["macro avg"]["f1-score"],
            "macro_precision": report["macro avg"]["precision"],
            "macro_recall": report["macro avg"]["recall"],
            "weighted_f1": report["weighted avg"]["f1-score"],
            "weighted_precision": report["weighted avg"]["precision"],
            "weighted_recall": report["weighted avg"]["recall"],
        })

statistical_methods = pd.DataFrame(statistical_methods_results)

Training on bow_text_no_lemma_stem_with_stopwords...
Training on tfidf_text_no_lemma_stem_with_stopwords...


In [27]:
statistical_methods

,variant,model,accuracy,macro_f1,macro_precision,macro_recall,weighted_f1,weighted_precision,weighted_recall
0,bow_text_no_lemma_stem_with_stopwords,SVC,0.820755,0.751468,0.768797,0.737156,0.817313,0.815842,0.820755
1,bow_text_no_lemma_stem_with_stopwords,XGB,0.830189,0.749985,0.830109,0.707580,0.819236,0.830418,0.830189
2,bow_text_no_lemma_stem_with_stopwords,LogisticRegression,0.821803,0.753013,0.761594,0.745494,0.819788,0.818403,0.821803
3,bow_text_no_lemma_stem_with_stopwords,KNN,0.699161,0.446410,0.771153,0.435460,0.622715,0.736911,0.699161
4,tfidf_text_no_lemma_stem_with_stopwords,SVC,0.827044,0.755249,0.798578,0.728004,0.820296,0.822640,0.827044
5,tfidf_text_no_lemma_stem_with_stopwords,XGB,0.814465,0.726235,0.825597,0.678027,0.800440,0.817435,0.814465
6,tfidf_text_no_lemma_stem_with_stopwords,LogisticRegression,0.824948,0.770459,0.758573,0.784475,0.827684,0.831976,0.824948
7,tfidf_text_no_lemma_stem_with_stopwords,KNN,0.690776,0.422575,0.790902,0.419796,0.606683,0.740076,0.690776


### 1.2 - Fixed Word Embedding Encoders

In [14]:
X_train_glove = npy_embeddings['glove_seq_text_no_lemma_stem_with_stopwords_train']
X_val_glove   = npy_embeddings['glove_seq_text_no_lemma_stem_with_stopwords_val']
X_test_glove  = npy_embeddings['glove_seq_text_no_lemma_stem_with_stopwords_test']

X_train_ft = npy_embeddings['ft_seq_text_no_lemma_stem_with_stopwords_train']
X_val_ft = npy_embeddings['ft_seq_text_no_lemma_stem_with_stopwords_val']
X_test_ft = npy_embeddings['ft_seq_text_no_lemma_stem_with_stopwords_test']

X_train_w2v = npy_embeddings['w2v_seq_text_no_lemma_stem_with_stopwords_train']
X_val_w2v = npy_embeddings['w2v_seq_text_no_lemma_stem_with_stopwords_val']
X_test_w2v = npy_embeddings['w2v_seq_text_no_lemma_stem_with_stopwords_test']

In [34]:
fixed_embedding_variants = {
    "glove":   (X_train_glove, X_val_glove),
    "ft":      (X_train_ft,    X_val_ft),
    "w2v":     (X_train_w2v,   X_val_w2v)
}

In [16]:
num_classes = 3

y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_val_cat = to_categorical(y_val, num_classes=num_classes)

In [ ]:
# Early stopping: stop if val_loss doesn't improve for 3 epochs
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=3, 
    restore_best_weights=True
)

callbacks = early_stopping

#### 1) BiLSTM

In [56]:
def build_bilstm(units, input_length, embed_size, num_classes):
    input_ = Input(shape=(input_length, embed_size))
    x = Bidirectional(LSTM(units, return_sequences=False, dropout=0.25, recurrent_dropout=0.25))(input_)
    out = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_, outputs=out)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

In [57]:
X_train_ft.shape , X_train_w2v.shape

((7634, 32, 200), (7634, 32, 200))

In [ ]:
results_fixed_encoders_bilstm = {}  

for name, (X_train, X_val) in fixed_embedding_variants.items():
    print(f"\nTraining and evaluating BiLSTM with {name.upper()} embeddings...")

    model = build_bilstm(
        units=64,
        input_length=X_train.shape[1],
        embed_size=X_train.shape[2],
        num_classes=num_classes
    )

    model.fit(
        X_train, y_train_cat,
        validation_data=(X_val, y_val_cat),
        epochs=100,  
        batch_size=32,
        callbacks=callbacks,
        verbose=1,
    )

    y_pred_probs = model.predict(X_val)
    y_pred = y_pred_probs.argmax(axis=1)
    y_val_argmax = y_val.argmax(axis=1) if y_val.ndim > 1 else y_val

    print(classification_report(y_val_argmax, y_pred, digits=4))

    
    results_fixed_encoders_bilstm[name] = classification_report(y_val_argmax, y_pred, digits=4, output_dict=True)


Training and evaluating BiLSTM with GLOVE embeddings...
Epoch 1/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 17s 43ms/step - accuracy: 0.6344 - loss: 0.8531 - val_accuracy: 0.7180 - val_loss: 0.6828
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.7241 - loss: 0.6769 - val_accuracy: 0.7453 - val_loss: 0.6016
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.7659 - loss: 0.5957 - val_accuracy: 0.7778 - val_loss: 0.5568
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - accuracy: 0.7804 - loss: 0.5486 - val_accuracy: 0.7987 - val_loss: 0.5314
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 12s 52ms/step - accuracy: 0.8059 - loss: 0.4924 - val_accuracy: 0.7956 - val_loss: 0.5169
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.8159 - loss: 0.4793 - val_accuracy: 0.8103 - val_loss: 0.4881
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 11s 47ms/step - accuracy: 0.8218 - loss: 0.4567 - val_accuracy: 0.8197 - val_loss: 0.4823
Epoch 8/100
239/239 ━━━━━━

c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined an

Epoch 1/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 24s 65ms/step - accuracy: 0.6314 - loss: 0.9055 - val_accuracy: 0.6551 - val_loss: 0.8645
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.6568 - loss: 0.8444 - val_accuracy: 0.6625 - val_loss: 0.8215
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 15s 62ms/step - accuracy: 0.6613 - loss: 0.8348 - val_accuracy: 0.6625 - val_loss: 0.8098
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - accuracy: 0.6736 - loss: 0.8036 - val_accuracy: 0.6709 - val_loss: 0.8007
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 10s 44ms/step - accuracy: 0.6769 - loss: 0.8029 - val_accuracy: 0.6677 - val_loss: 0.8029
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 11s 48ms/step - accuracy: 0.6708 - loss: 0.8021 - val_accuracy: 0.6698 - val_loss: 0.8007
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.6831 - loss: 0.7891 - val_accuracy: 0.6751 - val_loss: 0.7885
Epoch 8/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 11s 48ms/step - accuracy: 0.6723 - loss: 0

#### 2) BiLSTM + Attention Layer

In [ ]:
class SimpleAttention(Layer):
    def __init__(self, **kwargs):
        super(SimpleAttention, self).__init__(**kwargs)
        self.dense = Dense(1)

    def call(self, inputs):
        
        score = tf.nn.softmax(self.dense(inputs), axis=1)  
        context = tf.reduce_sum(score * inputs, axis=1)
        return context

In [ ]:
def build_bilstm_att(units, input_length, embed_size, num_classes=3):
    input_ = Input(shape=(input_length, embed_size)) 

    # BiLSTM with return_sequences=True so Attention can work on the sequence
    x = Bidirectional(LSTM(units, return_sequences=True, dropout=0.25, recurrent_dropout=0.25))(input_)
    
    # Attention layer
    x = SimpleAttention()(x)
    
    # Dense softmax output
    out = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_, outputs=out)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

In [ ]:
results_fixed_encoders_bilstm_att = {}  

for name, (X_train, X_val) in fixed_embedding_variants.items():
    print(f"\nTraining and evaluating BiLSTM with attention layer with {name.upper()} embeddings...")

    model = build_bilstm_att(
        units=64,
        input_length=X_train.shape[1],
        embed_size=X_train.shape[2],
        num_classes=num_classes
    )

    model.fit(
        X_train, y_train_cat,
        validation_data=(X_val, y_val_cat),
        epochs=100,  
        batch_size=32,
        callbacks=callbacks,
        verbose=1,
    )

    y_pred_probs = model.predict(X_val)
    y_pred = y_pred_probs.argmax(axis=1)
    y_val_argmax = y_val.argmax(axis=1) if y_val.ndim > 1 else y_val

    print(classification_report(y_val_argmax, y_pred, digits=4))

    results_fixed_encoders_bilstm_att[name] = classification_report(y_val_argmax, y_pred, digits=4, output_dict=True)


Training and evaluating BiLSTM with attention layer with GLOVE embeddings...
Epoch 1/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 18s 41ms/step - accuracy: 0.6622 - loss: 0.8514 - val_accuracy: 0.7390 - val_loss: 0.6435
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 9s 39ms/step - accuracy: 0.7369 - loss: 0.6525 - val_accuracy: 0.7851 - val_loss: 0.5524
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 9s 39ms/step - accuracy: 0.7808 - loss: 0.5690 - val_accuracy: 0.7904 - val_loss: 0.5301
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 14s 60ms/step - accuracy: 0.7995 - loss: 0.5162 - val_accuracy: 0.8103 - val_loss: 0.5054
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - accuracy: 0.8092 - loss: 0.4870 - val_accuracy: 0.8187 - val_loss: 0.4930
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - accuracy: 0.8292 - loss: 0.4452 - val_accuracy: 0.8092 - val_loss: 0.4960
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - accuracy: 0.8460 - loss: 0.4073 - val_accuracy: 0.8239 - val_loss: 0.4903
Epoch 8

c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined an

Epoch 1/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 18s 46ms/step - accuracy: 0.6364 - loss: 0.9043 - val_accuracy: 0.6478 - val_loss: 0.8605
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.6513 - loss: 0.8526 - val_accuracy: 0.6551 - val_loss: 0.8285
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.6627 - loss: 0.8326 - val_accuracy: 0.6551 - val_loss: 0.8164
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0.6669 - loss: 0.8102 - val_accuracy: 0.6593 - val_loss: 0.8155
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.6646 - loss: 0.8128 - val_accuracy: 0.6614 - val_loss: 0.8136
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.6596 - loss: 0.8196 - val_accuracy: 0.6635 - val_loss: 0.8091
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.6655 - loss: 0.8085 - val_accuracy: 0.6688 - val_loss: 0.7952
Epoch 8/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 11s 46ms/step - accuracy: 0.6726 - loss: 0.800

c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined an

#### 3) BiGRU

In [36]:
def build_bigru(units, input_length, embed_size, num_classes=3):
    input_ = Input(shape=(input_length, embed_size))
    x = Bidirectional(GRU(units, return_sequences=True, dropout=0.25, recurrent_dropout=0.25))(input_)
    x = Flatten()(x)
    out = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_, outputs=out)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

In [ ]:
results_fixed_encoders_bigru = {}

for name, (X_train, X_val) in fixed_embedding_variants.items():
    print(f"\nTraining and evaluating BiGRU with {name.upper()} embeddings...")

    model = build_bigru(
        units=64,
        input_length=X_train.shape[1],
        embed_size=X_train.shape[2],
        num_classes=num_classes
    )

    model.fit(
        X_train, y_train_cat,
        validation_data=(X_val, y_val_cat),
        epochs=100,  
        batch_size=32,
        callbacks=callbacks,
        verbose=1,
    )

    y_pred_probs = model.predict(X_val)
    y_pred = y_pred_probs.argmax(axis=1)
    y_val_argmax = y_val.argmax(axis=1) if y_val.ndim > 1 else y_val

    print(classification_report(y_val_argmax, y_pred, digits=4))

    results_fixed_encoders_bigru[name] = classification_report(y_val_argmax, y_pred, digits=4, output_dict=True)


Training and evaluating BiGRU with GLOVE embeddings...
Epoch 1/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 19s 47ms/step - accuracy: 0.6606 - loss: 0.8313 - val_accuracy: 0.7358 - val_loss: 0.6381
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.7668 - loss: 0.5984 - val_accuracy: 0.7673 - val_loss: 0.5873
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - accuracy: 0.7976 - loss: 0.5195 - val_accuracy: 0.7935 - val_loss: 0.5668
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.8162 - loss: 0.4623 - val_accuracy: 0.7925 - val_loss: 0.5611
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 13s 56ms/step - accuracy: 0.8523 - loss: 0.3909 - val_accuracy: 0.7977 - val_loss: 0.5711
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - accuracy: 0.8728 - loss: 0.3352 - val_accuracy: 0.8071 - val_loss: 0.5811
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.8881 - loss: 0.2906 - val_accuracy: 0.8029 - val_loss: 0.5850
30/30 ━━━━━━━━━━━━━━━━━━━━ 2

c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined an

239/239 ━━━━━━━━━━━━━━━━━━━━ 14s 40ms/step - accuracy: 0.6431 - loss: 0.9076 - val_accuracy: 0.6625 - val_loss: 0.8348
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 9s 39ms/step - accuracy: 0.6584 - loss: 0.8488 - val_accuracy: 0.6593 - val_loss: 0.8492
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.6651 - loss: 0.8273 - val_accuracy: 0.6677 - val_loss: 0.8109
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.6771 - loss: 0.8095 - val_accuracy: 0.6656 - val_loss: 0.8132
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - accuracy: 0.6663 - loss: 0.8143 - val_accuracy: 0.6688 - val_loss: 0.8036
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.6666 - loss: 0.8066 - val_accuracy: 0.6667 - val_loss: 0.7969
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 11s 47ms/step - accuracy: 0.6748 - loss: 0.7993 - val_accuracy: 0.6855 - val_loss: 0.8050
Epoch 8/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - accuracy: 0.6844 - loss: 0.7747 - val_a

#### 4) BiGRU + Attention Layer

In [38]:
def build_bigru_att(units, input_length, embed_size, num_classes=3):
    input_ = Input(shape=(input_length, embed_size))
    x = Bidirectional(GRU(units, return_sequences=True, dropout=0.25, recurrent_dropout=0.25))(input_)
    x = SimpleAttention()(x)
    out = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_, outputs=out)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

In [ ]:
results_fixed_encoders_bigru_att = {}

for name, (X_train, X_val) in fixed_embedding_variants.items():
    print(f"\nTraining and evaluating BiGRU with {name.upper()} embeddings...")

    model = build_bigru_att(
        units=64,
        input_length=X_train.shape[1],
        embed_size=X_train.shape[2],
        num_classes=num_classes
    )

    model.fit(
        X_train, y_train_cat,
        validation_data=(X_val, y_val_cat),
        epochs=100,  
        batch_size=32,
        callbacks=callbacks,
        verbose=1,
    )

    # Predict on validation set
    y_pred_probs = model.predict(X_val)
    y_pred = y_pred_probs.argmax(axis=1)
    y_val_argmax = y_val.argmax(axis=1) if y_val.ndim > 1 else y_val

    print(classification_report(y_val_argmax, y_pred, digits=4))

    results_fixed_encoders_bigru_att[name] = classification_report(y_val_argmax, y_pred, digits=4, output_dict=True)


Training and evaluating BiGRU with GLOVE embeddings...
Epoch 1/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 18s 50ms/step - accuracy: 0.6479 - loss: 0.8490 - val_accuracy: 0.7558 - val_loss: 0.6323
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - accuracy: 0.7653 - loss: 0.6158 - val_accuracy: 0.7820 - val_loss: 0.5539
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.7745 - loss: 0.5723 - val_accuracy: 0.7893 - val_loss: 0.5403
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 11s 46ms/step - accuracy: 0.8030 - loss: 0.5074 - val_accuracy: 0.8040 - val_loss: 0.5213
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 11s 47ms/step - accuracy: 0.8236 - loss: 0.4664 - val_accuracy: 0.8166 - val_loss: 0.5112
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 11s 47ms/step - accuracy: 0.8269 - loss: 0.4349 - val_accuracy: 0.8218 - val_loss: 0.4991
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 13s 54ms/step - accuracy: 0.8440 - loss: 0.4063 - val_accuracy: 0.8270 - val_loss: 0.4938
Epoch 8/100
239/239 ━━━━━━━

### 1.3 - Sentence Encoders

In [96]:
X_train_mini_lm = npy_embeddings['minilm_l12_v2_text_no_lemma_stem_with_stopwords_train']
X_val_mini_lm = npy_embeddings['minilm_l12_v2_text_no_lemma_stem_with_stopwords_val']
X_test_mini_lm = npy_embeddings['minilm_l12_v2_text_no_lemma_stem_with_stopwords_test']

X_train_mpnet = npy_embeddings['mpnet_base_v2_text_no_lemma_stem_with_stopwords_train']
X_val_mpnet = npy_embeddings['mpnet_base_v2_text_no_lemma_stem_with_stopwords_val']
X_test_mpnet = npy_embeddings['mpnet_base_v2_text_no_lemma_stem_with_stopwords_test']

X_train_distilroberta = npy_embeddings['distilroberta_v1_text_no_lemma_stem_with_stopwords_train']
X_val_distilroberta = npy_embeddings['distilroberta_v1_text_no_lemma_stem_with_stopwords_val']
X_test_distilroberta = npy_embeddings['distilroberta_v1_text_no_lemma_stem_with_stopwords_test']

X_train_allmini = npy_embeddings['all_minilm_l6_v2_text_no_lemma_stem_with_stopwords_train']
X_val_allmini = npy_embeddings['all_minilm_l6_v2_text_no_lemma_stem_with_stopwords_val']
X_test_allmini = npy_embeddings['all_minilm_l6_v2_text_no_lemma_stem_with_stopwords_test']

In [ ]:
sentence_encoders_variants = {
    "mpnet_base_v2":   (X_train_mini_lm, X_val_mini_lm),
    "distilroberta_v1":      (X_train_distilroberta,    X_val_distilroberta),
    "minilm_l12_v2":     (X_train_mini_lm,   X_val_mini_lm),
    "all_minilm_l6_v2":     (X_train_allmini,   X_val_allmini)
}

In [97]:
X_train_mini_lm.shape, X_train_mpnet.shape, X_train_distilroberta.shape, X_train_allmini.shape

((7634, 384), (7634, 768), (7634, 768), (7634, 384))

In [32]:
def create_mlp_model(embedding_dim, num_classes):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(embedding_dim,)),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [33]:
embedding_dim = 384

sentence_encoder_models = {
    "SVC": SVC(class_weight='balanced', random_state=42),
    "XGB": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=300, class_weight='balanced', random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "MLP": KerasClassifier(
        model=create_mlp_model, 
        embedding_dim=embedding_dim,   
        num_classes=num_classes,          
        epochs=100, batch_size=32, verbose=1,
        callbacks=callbacks
    )
}

In [ ]:
results_sentence_encoders = {} 

for name, (X_train, X_val) in sentence_encoders_variants.items():
    y_train_cls = y_train.argmax(axis=1) if y_train.ndim > 1 else y_train
    y_val_cls   = y_val.argmax(axis=1) if y_val.ndim > 1 else y_val

    for model_name, model in sentence_encoder_models.items():
        print(f"\nTraining and evaluating {model_name} with {name.upper()} embeddings...")

        if model_name == "MLP":
            model = KerasClassifier(
                model=create_mlp_model, 
                embedding_dim=X_train.shape[1], 
                num_classes=num_classes,
                epochs=100, batch_size=32, verbose=1, callbacks=callbacks
            )
        else:
            model = clone(model)

        model.fit(X_train, y_train_cls)
        y_pred = model.predict(X_val)
        print(classification_report(y_val_cls, y_pred, digits=4))
        results_sentence_encoders.setdefault(name, {})[model_name] = classification_report(
            y_val_cls, y_pred, digits=4, output_dict=True)


Training and evaluating SVC with MPNET_BASE_V2 embeddings...
              precision    recall  f1-score   support

           0     0.6646    0.7569    0.7078       144
           1     0.6147    0.6979    0.6537       192
           2     0.8986    0.8317    0.8639       618

    accuracy                         0.7935       954
   macro avg     0.7260    0.7622    0.7418       954
weighted avg     0.8061    0.7935    0.7980       954


Training and evaluating XGB with MPNET_BASE_V2 embeddings...
              precision    recall  f1-score   support

           0     0.7222    0.3611    0.4815       144
           1     0.6667    0.4792    0.5576       192
           2     0.7809    0.9401    0.8532       618

    accuracy                         0.7600       954
   macro avg     0.7233    0.5935    0.6307       954
weighted avg     0.7491    0.7600    0.7376       954


Training and evaluating LogisticRegression with MPNET_BASE_V2 embeddings...
              precision    recall  f1

c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


239/239 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6472 - loss: 0.8661
Epoch 2/100
 54/239 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7444 - loss: 0.6304

c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7499 - loss: 0.6247
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7821 - loss: 0.5610
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7863 - loss: 0.5459
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8211 - loss: 0.4719
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8257 - loss: 0.4449
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8586 - loss: 0.3877
Epoch 8/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8629 - loss: 0.3601
Epoch 9/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8750 - loss: 0.3379
Epoch 10/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8990 - loss: 0.2821
Epoch 11/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9033 - loss: 0.2686
Epoch 12/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9210 - loss: 0.2232
Epoch 13/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/ste

...

239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7487 - loss: 0.6170
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7767 - loss: 0.5644
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7958 - loss: 0.5235
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8188 - loss: 0.4784
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8343 - loss: 0.4377
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8470 - loss: 0.4144
Epoch 8/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8641 - loss: 0.3786
Epoch 9/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8805 - loss: 0.3399
Epoch 10/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8934 - loss: 0.2940
Epoch 11/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9061 - loss: 0.2646
Epoch 12/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9122 - loss: 0.2505
Epoch 13/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/ste

c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


239/239 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6581 - loss: 0.8573
Epoch 2/100
 39/239 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7531 - loss: 0.6397

c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7453 - loss: 0.6356
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7746 - loss: 0.5678
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8000 - loss: 0.5271
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8141 - loss: 0.4830
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8261 - loss: 0.4501
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8395 - loss: 0.4133
Epoch 8/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8645 - loss: 0.3582
Epoch 9/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8812 - loss: 0.3339
Epoch 10/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8884 - loss: 0.3035
Epoch 11/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9117 - loss: 0.2588
Epoch 12/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9118 - loss: 0.2508
Epoch 13/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/ste

### 1.4 - Transformers

#### 1) Feature Extraction

In [118]:
X_train_bert_large_uncased = npy_embeddings['bert-large-uncased_text_no_lemma_stem_with_stopwords_train_meanpooled']
X_val_bert_large_uncased = npy_embeddings['bert-large-uncased_text_no_lemma_stem_with_stopwords_val_meanpooled']
X_test_bert_large_uncased = npy_embeddings['bert-large-uncased_text_no_lemma_stem_with_stopwords_test_meanpooled']

X_train_roberta_large_uncased = npy_embeddings['roberta-large_text_no_lemma_stem_with_stopwords_train_meanpooled']
X_val_roberta_large_uncased = npy_embeddings['roberta-large_text_no_lemma_stem_with_stopwords_val_meanpooled']
X_test_roberta_large_uncased = npy_embeddings['roberta-large_text_no_lemma_stem_with_stopwords_test_meanpooled']

X_train_fb_bart_large = npy_embeddings['facebook-bart-large_text_no_lemma_stem_with_stopwords_train_meanpooled']
X_val_fb_bart_large = npy_embeddings['facebook-bart-large_text_no_lemma_stem_with_stopwords_val_meanpooled']
X_test_fb_bart_large = npy_embeddings['facebook-bart-large_text_no_lemma_stem_with_stopwords_test_meanpooled']

In [119]:
transformers_fe_variants = {
    "bert_large_uncased":   (X_train_bert_large_uncased, X_val_bert_large_uncased),
    "roberta_large_uncased":      (X_train_roberta_large_uncased,    X_val_roberta_large_uncased),
    "fb_bart_large":     (X_train_fb_bart_large,   X_val_fb_bart_large),
}

In [ ]:
results_transformers_fe = {} 

for name, (X_train, X_val) in transformers_fe_variants.items():
    y_train_cls = y_train.argmax(axis=1) if y_train.ndim > 1 else y_train
    y_val_cls   = y_val.argmax(axis=1) if y_val.ndim > 1 else y_val

    for model_name, model in sentence_encoder_models.items():
        print(f"\nTraining and evaluating {model_name} with {name.upper()} embeddings...")

        if model_name == "MLP":
            model = KerasClassifier(
                model=create_mlp_model, 
                embedding_dim=X_train.shape[1], 
                num_classes=num_classes,
                epochs=100, batch_size=32, verbose=1
            )
        else:
            model = clone(model)

        model.fit(X_train, y_train_cls)
        y_pred = model.predict(X_val)
        print(classification_report(y_val_cls, y_pred, digits=4))
        results_transformers_fe.setdefault(name, {})[model_name] = classification_report(
            y_val_cls, y_pred, digits=4, output_dict=True)


Training and evaluating SVC with BERT_LARGE_UNCASED embeddings...
              precision    recall  f1-score   support

           0     0.4305    0.6667    0.5232       144
           1     0.5316    0.6562    0.5874       192
           2     0.9008    0.7201    0.8004       618

    accuracy                         0.6992       954
   macro avg     0.6210    0.6810    0.6370       954
weighted avg     0.7555    0.6992    0.7157       954


Training and evaluating XGB with BERT_LARGE_UNCASED embeddings...
              precision    recall  f1-score   support

           0     0.7164    0.3333    0.4550       144
           1     0.6887    0.3802    0.4899       192
           2     0.7554    0.9547    0.8435       618

    accuracy                         0.7453       954
   macro avg     0.7202    0.5561    0.5961       954
weighted avg     0.7361    0.7453    0.7137       954


Training and evaluating LogisticRegression with BERT_LARGE_UNCASED embeddings...


c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0     0.4245    0.6250    0.5056       144
           1     0.5491    0.6406    0.5913       192
           2     0.8861    0.7427    0.8081       618

    accuracy                         0.7044       954
   macro avg     0.6199    0.6694    0.6350       954
weighted avg     0.7486    0.7044    0.7188       954


Training and evaluating KNN with BERT_LARGE_UNCASED embeddings...
              precision    recall  f1-score   support

           0     0.4488    0.3958    0.4207       144
           1     0.4649    0.4479    0.4562       192
           2     0.8084    0.8398    0.8238       618

    accuracy                         0.6939       954
   macro avg     0.5740    0.5612    0.5669       954
weighted avg     0.6850    0.6939    0.6890       954


Training and evaluating MLP with BERT_LARGE_UNCASED embeddings...
Epoch 1/100


c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


239/239 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6431 - loss: 0.8684
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6953 - loss: 0.7319
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7085 - loss: 0.6865
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7376 - loss: 0.6373
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7410 - loss: 0.6265
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7485 - loss: 0.6062
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7664 - loss: 0.5753
Epoch 8/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7679 - loss: 0.5708
Epoch 9/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7869 - loss: 0.5381
Epoch 10/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7749 - loss: 0.5347
Epoch 11/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7921 - loss: 0.5207
Epoch 12/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step

...

239/239 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6049 - loss: 0.9298
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7385 - loss: 0.6629
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7710 - loss: 0.5830
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7915 - loss: 0.5276
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7983 - loss: 0.5079
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8101 - loss: 0.4717
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8215 - loss: 0.4555
Epoch 8/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8250 - loss: 0.4439
Epoch 9/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8437 - loss: 0.4017
Epoch 10/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8378 - loss: 0.4064
Epoch 11/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8383 - loss: 0.4038
Epoch 12/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step

c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0     0.5372    0.7014    0.6084       144
           1     0.5921    0.7031    0.6429       192
           2     0.8941    0.7783    0.8322       618

    accuracy                         0.7516       954
   macro avg     0.6745    0.7276    0.6945       954
weighted avg     0.7794    0.7516    0.7603       954


Training and evaluating KNN with FB_BART_LARGE embeddings...
              precision    recall  f1-score   support

           0     0.4656    0.4236    0.4436       144
           1     0.5399    0.5990    0.5679       192
           2     0.8607    0.8495    0.8550       618

    accuracy                         0.7348       954
   macro avg     0.6221    0.6240    0.6222       954
weighted avg     0.7365    0.7348    0.7352       954


Training and evaluating MLP with FB_BART_LARGE embeddings...
Epoch 1/100


c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


239/239 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6589 - loss: 0.8327
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7549 - loss: 0.6003
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7982 - loss: 0.5182
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8071 - loss: 0.4926
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8194 - loss: 0.4659
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8239 - loss: 0.4270
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8393 - loss: 0.4053
Epoch 8/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8583 - loss: 0.3557
Epoch 9/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8632 - loss: 0.3532
Epoch 10/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8724 - loss: 0.3195
Epoch 11/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8874 - loss: 0.2948
Epoch 12/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step

#### 2) Fine Tunning BERT

In [ ]:
# Build HuggingFace datasets from NumPy arrays and label Series
train_df = Dataset.from_dict({
    "text_no_lemma_stem_with_stopwords": npy_embeddings["train_text_no_lemma_stem_with_stopwords"],
    "label": y_train.tolist()
})

val_df = Dataset.from_dict({
    "text_no_lemma_stem_with_stopwords": npy_embeddings["val_text_no_lemma_stem_with_stopwords"],
    "label": y_val.tolist()
})

test_df = Dataset.from_dict({
    "text_no_lemma_stem_with_stopwords": npy_embeddings["test_text_no_lemma_stem_with_stopwords"],
    "label": y_test.tolist()
})

# Disable tokenizer parallelism 
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Load model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3).to(device)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["text_no_lemma_stem_with_stopwords"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_df.map(tokenize_function, batched=True)
val_dataset = val_df.map(tokenize_function, batched=True)

train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")
columns_to_keep = ['input_ids', 'attention_mask', 'labels']
train_dataset.set_format(type="torch", columns=columns_to_keep)
val_dataset.set_format(type="torch", columns=columns_to_keep)


training_args = TrainingArguments(
    output_dir="./results_bert_base",
    num_train_epochs=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=20,
    disable_tqdm=False,
    logging_dir='./logs_bert_base',
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

preds = trainer.predict(val_dataset)
y_pred = np.argmax(preds.predictions, axis=1)
y_true = val_df["label"]  

print(classification_report(y_true, y_pred, digits=4))

Using device: mps


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 954/954 [00:00<00:00, 20138.94 examples/s]
/var/folders/vb/r_cbg99j0dj7dml3jnpfyl9h0000gn/T/ipykernel_1953/2019632396.py:79: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/Users/ricardo/miniconda3/envs/nlp_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,0.548800,0.454671
2,0.321900,0.542742
3,0.407600,0.785694
4,0.323200,0.760308


/Users/ricardo/miniconda3/envs/nlp_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/ricardo/miniconda3/envs/nlp_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/ricardo/miniconda3/envs/nlp_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/ricardo/miniconda3/envs/nlp_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_m

              precision    recall  f1-score   support

           0     0.7583    0.6319    0.6894       144
           1     0.7448    0.7448    0.7448       192
           2     0.8801    0.9142    0.8968       618

    accuracy                         0.8375       954
   macro avg     0.7944    0.7637    0.7770       954
weighted avg     0.8345    0.8375    0.8349       954



### 1.5 - Domain Specific Transformers

#### 1) Feature Extraction

In [23]:
X_train_finbert = npy_embeddings['finbert_text_no_lemma_stem_with_stopwords_train_meanpooled']
X_val_finbert = npy_embeddings['finbert_text_no_lemma_stem_with_stopwords_val_meanpooled']
X_test_finbert = npy_embeddings['finbert_text_no_lemma_stem_with_stopwords_test_meanpooled']

X_train_bertweet = npy_embeddings['berttweet_text_no_lemma_stem_with_stopwords_train_meanpooled']
X_val_bertweet = npy_embeddings['berttweet_text_no_lemma_stem_with_stopwords_val_meanpooled']
X_test_bertweet = npy_embeddings['berttweet_text_no_lemma_stem_with_stopwords_test_meanpooled']

X_train_fintwitbert = npy_embeddings['fintwitbert_text_no_lemma_stem_with_stopwords_train_meanpooled']
X_val_fintwitbert = npy_embeddings['fintwitbert_text_no_lemma_stem_with_stopwords_val_meanpooled']
X_test_fintwitbert = npy_embeddings['fintwitbert_text_no_lemma_stem_with_stopwords_test_meanpooled']

In [24]:
domain_transformers_fe_variants = {
    "finbert":   (X_train_finbert, X_val_finbert),
    "bert_tweet":      (X_train_bertweet,    X_val_bertweet),
    "fintwitbert":     (X_train_fintwitbert,   X_val_fintwitbert),
}

In [ ]:
results_domain_transformers_fe = {} 

for name, (X_train, X_val) in domain_transformers_fe_variants.items():
    y_train_cls = y_train.argmax(axis=1) if y_train.ndim > 1 else y_train
    y_val_cls   = y_val.argmax(axis=1) if y_val.ndim > 1 else y_val

    for model_name, model in sentence_encoder_models.items():
        print(f"\nTraining and evaluating {model_name} with {name.upper()} embeddings...")

        if model_name == "MLP":
            model = KerasClassifier(
                model=create_mlp_model, 
                embedding_dim=X_train.shape[1], 
                num_classes=num_classes,
                epochs=100, batch_size=32, verbose=1
            )
        else:
            model = clone(model)

        model.fit(X_train, y_train_cls)
        y_pred = model.predict(X_val)
        print(classification_report(y_val_cls, y_pred, digits=4))
        results_domain_transformers_fe.setdefault(name, {})[model_name] = classification_report(
            y_val_cls, y_pred, digits=4, output_dict=True)


Training and evaluating SVC with FINBERT embeddings...
              precision    recall  f1-score   support

           0     0.5714    0.7500    0.6486       144
           1     0.6186    0.6927    0.6536       192
           2     0.8982    0.7994    0.8459       618

    accuracy                         0.7704       954
   macro avg     0.6961    0.7474    0.7160       954
weighted avg     0.7926    0.7704    0.7774       954


Training and evaluating XGB with FINBERT embeddings...
              precision    recall  f1-score   support

           0     0.7732    0.5208    0.6224       144
           1     0.6993    0.5573    0.6203       192
           2     0.8239    0.9385    0.8775       618

    accuracy                         0.7987       954
   macro avg     0.7655    0.6722    0.7067       954
weighted avg     0.7912    0.7987    0.7872       954


Training and evaluating LogisticRegression with FINBERT embeddings...
              precision    recall  f1-score   support



c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6702 - loss: 0.7706
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7423 - loss: 0.6435
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7543 - loss: 0.6242
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7675 - loss: 0.5945
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7730 - loss: 0.5688
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7903 - loss: 0.5496
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8007 - loss: 0.5329
Epoch 8/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8072 - loss: 0.5124
Epoch 9/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8086 - loss: 0.4957
Epoch 10/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8160 - loss: 0.4836
Epoch 11/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8241 - loss: 0.4605
Epoch 12/100
239/239 ━━━━━━━━━━━━━━━━━━━━

c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0     0.5217    0.7500    0.6154       144
           1     0.6018    0.7083    0.6507       192
           2     0.9117    0.7686    0.8341       618

    accuracy                         0.7537       954
   macro avg     0.6784    0.7423    0.7001       954
weighted avg     0.7905    0.7537    0.7642       954


Training and evaluating KNN with BERT_TWEET embeddings...
              precision    recall  f1-score   support

           0     0.4865    0.5000    0.4932       144
           1     0.5484    0.5312    0.5397       192
           2     0.8403    0.8430    0.8417       618

    accuracy                         0.7285       954
   macro avg     0.6251    0.6248    0.6248       954
weighted avg     0.7282    0.7285    0.7283       954


Training and evaluating MLP with BERT_TWEET embeddings...
Epoch 1/100


c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


239/239 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6621 - loss: 0.8191
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7247 - loss: 0.6674
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7515 - loss: 0.6051
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7627 - loss: 0.5735
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7770 - loss: 0.5508
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7855 - loss: 0.5298
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7906 - loss: 0.5070
Epoch 8/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8069 - loss: 0.4891
Epoch 9/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8086 - loss: 0.4752
Epoch 10/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8192 - loss: 0.4632
Epoch 11/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8197 - loss: 0.4401
Epoch 12/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


              precision    recall  f1-score   support

           0     0.4829    0.6875    0.5673       144
           1     0.5588    0.6927    0.6186       192
           2     0.8787    0.7265    0.7954       618

    accuracy                         0.7138       954
   macro avg     0.6401    0.7022    0.6604       954
weighted avg     0.7546    0.7138    0.7254       954


Training and evaluating KNN with FINTWITBERT embeddings...
              precision    recall  f1-score   support

           0     0.5944    0.5903    0.5923       144
           1     0.7115    0.5781    0.6379       192
           2     0.8244    0.8738    0.8484       618

    accuracy                         0.7715       954
   macro avg     0.7101    0.6807    0.6929       954
weighted avg     0.7670    0.7715    0.7674       954


Training and evaluating MLP with FINTWITBERT embeddings...
Epoch 1/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6729 - loss: 0.7815
Epoch 2/100
239/239 ━━━━━━━━━━━

#### 2) Fine Tunning -> FinTwit BERT , BART 

Fine Tunning FinTwit BERT ( encoder)

In [ ]:
# Disable tokenizer parallelism
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Device
print(f"Using device: {device}")

# Load model and tokenizer
model_name = "StephanAkkerman/FinTwitBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3).to(device)

def tokenize_function(examples):
    return tokenizer(
        examples["text_no_lemma_stem_with_stopwords"],
        padding="max_length",
        truncation=True,
        max_length=64
    )
    
train_dataset = train_df.map(tokenize_function, batched=True)
val_dataset = val_df.map(tokenize_function, batched=True)

train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")
columns_to_keep = ['input_ids', 'attention_mask', 'labels']
train_dataset.set_format(type="torch", columns=columns_to_keep)
val_dataset.set_format(type="torch", columns=columns_to_keep)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

preds = trainer.predict(val_dataset)
y_pred = np.argmax(preds.predictions, axis=1)
y_true = val_df["label"] 

print(classification_report(y_true, y_pred, digits=4))

Using device: mps


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at StephanAkkerman/FinTwitBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 954/954 [00:00<00:00, 22288.26 examples/s]
/var/folders/vb/r_cbg99j0dj7dml3jnpfyl9h0000gn/T/ipykernel_1953/2061078254.py:75: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/Users/ricardo/miniconda3/envs/nlp_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,0.584000,0.567262
2,0.319500,0.616466
3,0.219500,0.696817
4,0.144700,0.644307


/Users/ricardo/miniconda3/envs/nlp_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/ricardo/miniconda3/envs/nlp_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/ricardo/miniconda3/envs/nlp_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/ricardo/miniconda3/envs/nlp_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_m

              precision    recall  f1-score   support

           0     0.7542    0.6181    0.6794       144
           1     0.8662    0.6406    0.7365       192
           2     0.8473    0.9515    0.8963       618

    accuracy                         0.8386       954
   macro avg     0.8226    0.7367    0.7708       954
weighted avg     0.8370    0.8386    0.8314       954



Fine tuning BART (encoder-decoder)

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

print(f"Using device: {device}")

model_name = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3).to(device)

def tokenize_function(examples):
    return tokenizer(
        examples["text_no_lemma_stem_with_stopwords"],
        padding="max_length",
        truncation=True,
        max_length=64
    )
    
train_dataset = train_df.map(tokenize_function, batched=True)
val_dataset = val_df.map(tokenize_function, batched=True)

train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")
columns_to_keep = ['input_ids', 'attention_mask', 'labels']
train_dataset.set_format(type="torch", columns=columns_to_keep)
val_dataset.set_format(type="torch", columns=columns_to_keep)

training_args = TrainingArguments(
    output_dir="./results_BART",
    num_train_epochs=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs_BART',
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()


preds = trainer.predict(val_dataset)
logits = preds.predictions[0] if isinstance(preds.predictions, tuple) else preds.predictions
y_pred = np.argmax(logits, axis=1)
y_true = np.array(val_df["label"])

print(classification_report(y_true, y_pred, digits=4))

Using device: mps


Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 954/954 [00:00<00:00, 25256.05 examples/s]
/var/folders/vb/r_cbg99j0dj7dml3jnpfyl9h0000gn/T/ipykernel_1953/3851088187.py:74: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/Users/ricardo/miniconda3/envs/nlp_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,0.681900,0.552077
2,0.478800,0.479978
3,0.364200,0.626906
4,0.309800,0.726914
5,0.241500,0.740788


/Users/ricardo/miniconda3/envs/nlp_env/lib/python3.10/site-packages/transformers/configuration_utils.py:394: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}
  warnings.warn(
/Users/ricardo/miniconda3/envs/nlp_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/ricardo/miniconda3/envs/nlp_env/lib/python3.10/site-packages/transformers/configuration_utils.py:394: UserWarning: Some n

              precision    recall  f1-score   support

           0     0.7731    0.6389    0.6996       144
           1     0.8000    0.7083    0.7514       192
           2     0.8647    0.9304    0.8963       618

    accuracy                         0.8417       954
   macro avg     0.8126    0.7592    0.7824       954
weighted avg     0.8378    0.8417    0.8375       954



### 2 - Hyperparameter Tuning 

We decided to perform hyperparameter tuning only to our best model that it is not a transformer, due to computational limitations. Nonetheless, we recognize that in an ideal experiment we should have done hyperparameter tuning for each model and its respective embedding technique.

1) Try different embedding dimensions for GLOVE-Twitter 

In [37]:
X_train_glove_100d = npy_embeddings["glove_seq_text_no_lemma_stem_with_stopwords_100d_train"]
X_val_glove_100d = npy_embeddings["glove_seq_text_no_lemma_stem_with_stopwords_100d_val"]

X_train_glove_50d = npy_embeddings["glove_seq_text_no_lemma_stem_with_stopwords_50d_train"]
X_val_glove_50d = npy_embeddings["glove_seq_text_no_lemma_stem_with_stopwords_50d_val"]

X_train_glove_25d = npy_embeddings["glove_seq_text_no_lemma_stem_with_stopwords_25d_train"]
X_val_glove_25d = npy_embeddings["glove_seq_text_no_lemma_stem_with_stopwords_25d_val"]

In [ ]:
glove_dimensions_variants = {
    "glove_100d":   (X_train_glove_100d, X_val_glove_100d),
    "glove_50d":      (X_train_glove_50d,    X_val_glove_50d),
    "glove_25d":     (X_train_glove_25d,   X_val_glove_25d),
}

In [ ]:
results_glove_dimensions_bigru_att = {}

for name, (X_train, X_val) in glove_dimensions_variants.items():
    print(f"\nTraining and evaluating BiGRU with {name.upper()} embeddings...")

    model = build_bigru_att(
        units=64,
        input_length=X_train.shape[1],
        embed_size=X_train.shape[2],
        num_classes=num_classes
    )

    model.fit(
        X_train, y_train_cat,
        validation_data=(X_val, y_val_cat),
        epochs=100,
        batch_size=32,
        callbacks=callbacks,
        verbose=1,
    )

    y_pred_probs = model.predict(X_val)
    y_pred = y_pred_probs.argmax(axis=1)
    y_val_argmax = y_val.argmax(axis=1) if y_val.ndim > 1 else y_val

    print(classification_report(y_val_argmax, y_pred, digits=4))

    results_glove_dimensions_bigru_att[name] = classification_report(y_val_argmax, y_pred, digits=4, output_dict=True)


Training and evaluating BiGRU with GLOVE_200D embeddings...

Epoch 1/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 23s 67ms/step - accuracy: 0.6443 - loss: 0.8496 - val_accuracy: 0.7589 - val_loss: 0.6351
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.7517 - loss: 0.6366 - val_accuracy: 0.7851 - val_loss: 0.5553
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.7777 - loss: 0.5701 - val_accuracy: 0.8019 - val_loss: 0.5243
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.7958 - loss: 0.5192 - val_accuracy: 0.8124 - val_loss: 0.5069
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.8183 - loss: 0.4803 - val_accuracy: 0.8166 - val_loss: 0.5016
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.8280 - loss: 0.4485 - val_accuracy: 0.8197 - val_loss: 0.4901
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.8446 - loss: 0.4137 - val_accuracy: 0.8187 - val_loss: 0.4895
Epoch 8/100
239/239 ━━━━━━

2) Test different attention layer configurations

2.1 - Bahdanau (Additive) Attention

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, values):
        score = tf.nn.tanh(self.W1(values))  
        attention_weights = tf.nn.softmax(self.V(score), axis=1)  
        context_vector = attention_weights * values  
        context_vector = tf.reduce_sum(context_vector, axis=1)  
        return context_vector

In [ ]:
def build_bigru_bahdanau_att(units, input_length, embed_size, num_classes=3):
    input_ = Input(shape=(input_length, embed_size))
    x = Bidirectional(GRU(units, return_sequences=True, dropout=0.25, recurrent_dropout=0.25))(input_)
    x = BahdanauAttention(units)(x) 
    out = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_, outputs=out)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
results_bigru_bahdanau_att = {}

print(f"\nTraining and evaluating BiGRU + Bahdanau Attention with GloVe embeddings...")

model = build_bigru_bahdanau_att(
    units=64,
    input_length=X_train_glove.shape[1],
    embed_size=X_train_glove.shape[2],
    num_classes=num_classes
)

model.fit(
    X_train_glove, y_train_cat,
    validation_data=(X_val_glove, y_val_cat),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1,
)

y_pred_probs = model.predict(X_val_glove)
y_pred = y_pred_probs.argmax(axis=1)
y_val_argmax = y_val.argmax(axis=1) if y_val.ndim > 1 else y_val

print(classification_report(y_val_argmax, y_pred, digits=4))

results_bigru_bahdanau_att["glove"] = classification_report(
    y_val_argmax, y_pred, digits=4, output_dict=True
)



Training and evaluating BiGRU + Bahdanau Attention with GloVe embeddings...
Epoch 1/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - accuracy: 0.6506 - loss: 0.8427 - val_accuracy: 0.7600 - val_loss: 0.6189
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - accuracy: 0.7490 - loss: 0.6407 - val_accuracy: 0.7830 - val_loss: 0.5671
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 14s 58ms/step - accuracy: 0.7791 - loss: 0.5746 - val_accuracy: 0.7966 - val_loss: 0.5304
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 15s 61ms/step - accuracy: 0.7989 - loss: 0.5341 - val_accuracy: 0.8113 - val_loss: 0.5163
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.8122 - loss: 0.4886 - val_accuracy: 0.8061 - val_loss: 0.5146
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.8291 - loss: 0.4583 - val_accuracy: 0.8239 - val_loss: 0.5036
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0.8323 - loss: 0.4276 - val_accuracy: 0.8291 - val_loss: 0.4907
Epoch 8/1

2.2 - Multi-Head Self-Attention 

In [ ]:
class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)

    def call(self, inputs):
        attn_output = self.mha(inputs, inputs) 
        return tf.reduce_mean(attn_output, axis=1)

In [56]:
def build_bigru_multiheadself_att(units, input_length, embed_size, num_classes=3):
    input_ = Input(shape=(input_length, embed_size))
    x = Bidirectional(GRU(units, return_sequences=True, dropout=0.25, recurrent_dropout=0.25))(input_)
    x = MultiHeadSelfAttention(units)(x) 
    out = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_, outputs=out)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
results_bigru_multiheadself_att = {}

print(f"\nTraining and evaluating BiGRU + Multi Head Self Attention with GloVe embeddings...")

model = build_bigru_multiheadself_att(
    units=64,
    input_length=X_train_glove.shape[1],
    embed_size=X_train_glove.shape[2],
    num_classes=num_classes
)

model.fit(
    X_train_glove, y_train_cat,
    validation_data=(X_val_glove, y_val_cat),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1,
)

y_pred_probs = model.predict(X_val_glove)
y_pred = y_pred_probs.argmax(axis=1)
y_val_argmax = y_val.argmax(axis=1) if y_val.ndim > 1 else y_val

print(classification_report(y_val_argmax, y_pred, digits=4))

results_bigru_multiheadself_att["glove"] = classification_report(
    y_val_argmax, y_pred, digits=4, output_dict=True
)


Training and evaluating BiGRU + Multi Head Self Attention with GloVe embeddings...
Epoch 1/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 28s 84ms/step - accuracy: 0.6477 - loss: 0.8447 - val_accuracy: 0.7201 - val_loss: 0.6542
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 18s 76ms/step - accuracy: 0.7283 - loss: 0.6791 - val_accuracy: 0.7736 - val_loss: 0.5704
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 19s 78ms/step - accuracy: 0.7685 - loss: 0.5955 - val_accuracy: 0.7893 - val_loss: 0.5402
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 17s 71ms/step - accuracy: 0.7945 - loss: 0.5350 - val_accuracy: 0.8050 - val_loss: 0.5165
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 18s 74ms/step - accuracy: 0.8066 - loss: 0.4963 - val_accuracy: 0.8197 - val_loss: 0.5023
Epoch 6/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 21s 88ms/step - accuracy: 0.8324 - loss: 0.4417 - val_accuracy: 0.8113 - val_loss: 0.5036
Epoch 7/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 21s 87ms/step - accuracy: 0.8429 - loss: 0.4100 - val_accuracy: 0.8249 - val_loss: 0.4987

3) Test different hyperparameters configurations

In [43]:
def macro_f1(y_true, y_pred):
    num_classes = tf.shape(y_true)[-1]
    y_pred = K.one_hot(K.argmax(y_pred, axis=-1), num_classes)
    y_true = K.cast(y_true, tf.float32)
    tp = K.sum(y_true * y_pred, axis=0)
    fp = K.sum(y_pred, axis=0) - tp
    fn = K.sum(y_true, axis=0) - tp
    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    macro_f1 = K.mean(f1)
    return macro_f1

In [44]:
def build_bigru_tuner(hp):
    input_ = Input(shape=(input_length, embed_size))
    x = Dropout(hp.Choice('embedding_dropout', [0.0, 0.2]))(input_)

    for i in range(hp.Choice('num_layers', [1, 2])):
        return_sequences = (i != hp.get('num_layers') - 1)
        x = Bidirectional(
            GRU(
                units=hp.Choice('units', [64, 128]),
                return_sequences=return_sequences,
                dropout=hp.Choice('dropout', [0.2, 0.4]),
                recurrent_dropout=hp.Choice('recurrent_dropout', [0.2, 0.4]),
                kernel_regularizer=tf.keras.regularizers.l2(hp.Choice('l2', [0.0, 1e-4]))
            )
        )(x)

    if hp.Boolean('extra_dense'):
        x = Dense(
            64, activation='relu',
            kernel_regularizer=tf.keras.regularizers.l2(hp.Choice('l2', [0.0, 1e-4]))
        )(x)
        x = Dropout(0.25)(x)

    out = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_, outputs=out)

    optimizer_choice = hp.Choice('optimizer', ['adam', 'rmsprop', 'nadam'])
    learning_rate = hp.Choice('learning_rate', [1e-3, 5e-4])
    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer_choice == 'nadam':
        optimizer = tf.keras.optimizers.Nadam(learning_rate=learning_rate)

    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy', macro_f1]
    )
    return model

In [46]:
input_length = X_train_glove.shape[1]
embed_size = X_train_glove.shape[2]
num_classes = y_train_cat.shape[1]

tuner = kt.BayesianOptimization(
    build_bigru_tuner,
    objective=kt.Objective("val_macro_f1", direction="max"),
    max_trials=50, 
    directory='tuner_dir',
    project_name='bigru_att'
)


In [48]:
tuner.search(
    X_train_glove, y_train_cat,
    validation_data=(X_val_glove, y_val_cat),
    epochs=100,
    batch_size=32,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)],
    verbose=1
)

Trial 50 Complete [00h 18m 45s]
val_macro_f1: 0.7721172571182251

Best val_macro_f1 So Far: 0.7761452794075012
Total elapsed time: 11h 22m 24s


In [49]:
best_hps = tuner.get_best_hyperparameters(1)[0]
best_model = tuner.get_best_models(1)[0]
print("Best hyperparameters:", best_hps.values)

Best hyperparameters: {'embedding_dropout': 0.0, 'num_layers': 1, 'units': 128, 'dropout': 0.4, 'recurrent_dropout': 0.2, 'l2': 0.0, 'extra_dense': True, 'optimizer': 'nadam', 'learning_rate': 0.0005}


c:\Users\alexg\Ambiente de Trabalho\Mestrado Data Science NOVA IMS\2nd semester\Text Mining\tm_venv\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 23 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [51]:
def build_bigru_att(
    units,
    input_length,
    embed_size,
    num_classes=3,
    embedding_dropout=0.0,
    dropout=0.25,
    recurrent_dropout=0.25,
    l2=0.0,
    extra_dense=False,
    optimizer='adam',
    learning_rate=0.001
):

    input_ = Input(shape=(input_length, embed_size))
    x = Dropout(embedding_dropout)(input_)

    x = Bidirectional(GRU(
        units, 
        return_sequences=True, 
        dropout=dropout, 
        recurrent_dropout=recurrent_dropout, 
        kernel_regularizer=regularizers.l2(l2)
    ))(x)
    x = SimpleAttention()(x)

    if extra_dense:
        x = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(l2))(x)
        x = Dropout(0.25)(x)

    out = Dense(num_classes, activation='softmax')(x)

    # Set optimizer
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'nadam':
        opt = Nadam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    else:
        raise ValueError("Unsupported optimizer.")

    model = Model(inputs=input_, outputs=out)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model


In [ ]:
tuned_model = build_bigru_att(
    units=best_hps['units'],
    input_length=input_length,
    embed_size=embed_size,
    num_classes=3,
    embedding_dropout=best_hps['embedding_dropout'],
    dropout=best_hps['dropout'],
    recurrent_dropout=best_hps['recurrent_dropout'],
    l2=best_hps['l2'],
    extra_dense=best_hps['extra_dense'],
    optimizer=best_hps['optimizer'],
    learning_rate=best_hps['learning_rate']
)


In [ ]:
results_bigru_att_tuned_glove = {}

print(f"\nTraining and evaluating Tuned BiGRU with GLOVE embeddings...")

tuned_model.fit(
    X_train_glove, y_train_cat,
    validation_data=(X_val_glove, y_val_cat),
    epochs=100,  
    batch_size=32,
    callbacks=callbacks,
    verbose=1,
)

y_pred_probs = tuned_model.predict(X_val_glove)
y_pred = y_pred_probs.argmax(axis=1)
y_val_argmax = y_val.argmax(axis=1) if y_val.ndim > 1 else y_val

print(classification_report(y_val_argmax, y_pred, digits=4))

results_bigru_att_tuned_glove["glove"] = classification_report(y_val_argmax, y_pred, digits=4, output_dict=True)



Training and evaluating Tuned BiGRU with GLOVE embeddings...
Epoch 1/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 90s 359ms/step - accuracy: 0.8465 - loss: 0.4847 - val_accuracy: 0.8208 - val_loss: 0.5799
Epoch 2/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 91s 380ms/step - accuracy: 0.8593 - loss: 0.4457 - val_accuracy: 0.8344 - val_loss: 0.5771
Epoch 3/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 73s 306ms/step - accuracy: 0.8759 - loss: 0.3969 - val_accuracy: 0.8291 - val_loss: 0.5772
Epoch 4/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 82s 342ms/step - accuracy: 0.8879 - loss: 0.3851 - val_accuracy: 0.8270 - val_loss: 0.6128
Epoch 5/100
239/239 ━━━━━━━━━━━━━━━━━━━━ 85s 356ms/step - accuracy: 0.8937 - loss: 0.3676 - val_accuracy: 0.8239 - val_loss: 0.6064
30/30 ━━━━━━━━━━━━━━━━━━━━ 7s 209ms/step
              precision    recall  f1-score   support

           0     0.7154    0.6458    0.6788       144
           1     0.7803    0.7031    0.7397       192
           2     0.8725    0.9191    0.8952       618

    accuracy       